The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.

Number of reviews: 568,454
Number of users: 256,059
Number of products: 74,258
Timespan: Oct 1999 - Oct 2012
Number of Attributes/Columns in data: 10

Attribute Information:

Id
ProductId - unique identifier for the product
UserId - unqiue identifier for the user
ProfileName
HelpfulnessNumerator - number of users who found the review helpful
HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
Score - rating between 1 and 5
Time - timestamp for the review
Summary - brief summary of the review
Text - text of the review
Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).


[Q] How to determine if a review is positive or negative?

[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.

we will not be using the rating as a measuremt because then it would be very easy to identify whether the review is positive or not using a simple if-else condition , however we would be using ot to test our model


In [4]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
import seaborn as sns
import re

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [38]:
# using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite')
#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT *
FROM reviews
WHERE Score!=3
""",con)

# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x > 3:
        return "positive"
    return "negative"

#the below lines are to make the score column return positive and negative instead of the scores 1,2,4,5
actual_score = filtered_data["Score"]
positivenegative = actual_score.map(partition)
filtered_data["Score"] = positivenegative

#checking the number of atributes and size of my data
print("no of data points in our data :",filtered_data.shape)
filtered_data.head(3)


no of data points in our data : (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


## Exploratory Data Analysis


### Data Cleaning: Deduplication
It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.

In [39]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text,Count(*)
FROM Reviews
GROUP BY Userid
HAVING Count(*)>1
""",con)
print(display.shape)
display.head()

#almost 80k entries that are repeated by the same user

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,Count(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [40]:
#one such example
display = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score!=3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductId
""",con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [41]:
#sorting the df according to accending productid

sorted_data = filtered_data.sort_values("ProductId",axis = 0,ascending=True)

In [42]:
#removing the duplicate values
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Score","Time","Summary","Text"} , keep = "first" , inplace = False)
final.shape


(366392, 10)

In [43]:
#HelpfulnessNumerator can never be greater than HelpfulnessDenominator
display = pd.read_sql_query("""
SELECT Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
FROM Reviews
WHERE HelpfulnessNumerator>HelpfulnessDenominator
ORDER BY ProductId
""",con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [44]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
final.shape


(366390, 10)

In [45]:
final["Score"].value_counts()

positive    308679
negative     57711
Name: Score, dtype: int64

In [46]:
stop = stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


## Bag Of Words

In [47]:

count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final["Text"])

In [48]:
print(final_counts.shape)
type(final_counts)

(366390, 115281)


scipy.sparse.csr.csr_matrix

## Data preprocessing 

In [49]:
# removing html tags and punctuations from the reviews


sno = nltk.stem.SnowballStemmer("english") #initialising the snowball stemmer

def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr," ", sentence)
    return cleantext

def cleanpunct(sentence):
    cleaned = re.sub('[.|,|!|,|)|(|/|\|\"|\'|#|@|$|-|%|]'," ",sentence)
    return cleaned


print(stop)    
print("***************")
print(sno.stem("tasty"))






{'that', 'shouldn', 'we', 'those', 'to', "couldn't", 'hasn', 'weren', 'her', 'myself', 'been', 'had', 'am', 'where', 'hadn', 'out', "haven't", 'into', 'which', 'haven', 'he', 'did', 'during', 'then', 'before', 'was', 'being', 'any', 'until', 'don', 'has', 'd', 'yourself', "she's", 'wouldn', 'shan', 'so', 'these', 'him', "needn't", 'very', 'who', 'such', 'only', "you're", 'should', 'against', "mightn't", "hasn't", 'were', 'again', 'over', 'above', 'how', 've', 'ma', 'each', 'now', 'theirs', "hadn't", 'do', 'mightn', 'the', 'ours', 'isn', 'his', 'they', 'at', 'won', "shan't", 'our', 'them', 'their', 'couldn', 'm', 'themselves', 'below', 'br', 'hers', 'an', 'most', 'this', 'will', 'and', 'if', "won't", "you've", 'there', 's', 't', 'having', 'between', 'yours', 'have', "wouldn't", 'doesn', 'once', 'be', 'few', 'on', "didn't", 'some', 'because', 'through', 'it', 'than', 'ourselves', 'why', 'y', 'needn', 'or', 'aren', 'your', 'for', 'more', 'doing', 'ain', 'she', 'from', 'i', 'me', 'himself'

In [36]:
# this will take time to run as it is working on a lot of values

i = 0
all_positive_words = []
final_string = []
all_negative_words = []
str_1 = ""
for sent in final["Text"].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunct(w).split():
            if cleaned_words.isalpha() and len(cleaned_words) > 2 :
                if cleaned_words.lower() not in stop:
                    s = sno.stem(cleaned_words.lower()).encode("utf8")
                    filtered_sentence.append(s)
                    if (final["Score"].values)[i] == "positive":
                        all_positive_words.append(s) #all positive words
                    if (final["Score"].values)[i] == "negative":
                        all_negative_words.append(s) #all negative words
                else:
                    continue
            else:
                continue
                               
    str1 = b"".join(filtered_sentence) 
    final_string.append(str1)
    i+=1         

In [37]:
print(len(filtered_sentence))
print(len(final_string))
print(len(str1))

29
366390
126


In [38]:
final["CleanedText"] = final_string

In [39]:
conn = sqlite3.connect('final.sqlite')
c = conn.cursor() #you need cursor to fetch results  , by default it is at the first row
conn.text_factory = str # means that the text fields would be stored as strings
final.to_sql("Reviews" , conn , schema = None , if_exists = "replace")

## Bi grams and n grams

In [20]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)

print("the most common positive words are : " ,freq_dist_positive.most_common(20))
print("the most common negative words are : " ,freq_dist_negative.most_common(20))

the most common positive words are :  [(b'like', 140349), (b'tast', 129792), (b'good', 113419), (b'flavor', 110603), (b'love', 107965), (b'use', 104664), (b'great', 104495), (b'one', 97409), (b'product', 91542), (b'tri', 87365), (b'tea', 84974), (b'coffe', 79254), (b'make', 75593), (b'get', 72657), (b'food', 65621), (b'would', 56125), (b'time', 55556), (b'buy', 54512), (b'realli', 52967), (b'eat', 52349)]
the most common negative words are :  [(b'tast', 34843), (b'like', 32693), (b'product', 28710), (b'one', 20883), (b'flavor', 19780), (b'would', 18416), (b'tri', 17965), (b'use', 15637), (b'good', 15200), (b'coffe', 14793), (b'buy', 14037), (b'get', 13979), (b'food', 13600), (b'order', 12980), (b'tea', 11676), (b'even', 11314), (b'box', 10901), (b'amazon', 10116), (b'make', 10084), (b'eat', 9970)]


### observation :
it was observed that the words occuring in positive reviews are also occuring in negative reviews , so it is better to have bigrams and n grams instead of unigrams

In [40]:


count_vect = CountVectorizer(ngram_range = (1,2))
final_bigram_counts = count_vect.fit_transform(final["CleanedText"])

In [25]:
final_bigram_counts.shape
#here the dimensions are way more than bag of words which are basically unigrams . Hence, it is a better approach

(366390, 2910192)

## Using TF-IDF

In [41]:
tf_idf_vect = TfidfVectorizer(ngram_range = (1,2))
final_tf_idf = tf_idf_vect.fit_transform(final["CleanedText"])

In [33]:
final_tf_idf.shape

(366390, 2910192)

## Using the most import word2vec

In [51]:
import gensim
list_of_sent = []
for sent in final["Text"]:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunct(w).split():
            if cleaned_words.isalpha() :
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)
        
    
    

In [52]:
print(final["Text"][0])
print(list_of_sent[0])

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'we', 're', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'he', 's', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is', 'in', 'college']


In [57]:
w2v_model = gensim.models.Word2Vec(list_of_sent , min_count = 5 , vector_size = 50 , workers = 4)

In [59]:
w2v_model.wv.most_similar("tasty")

[('tastey', 0.9093310236930847),
 ('yummy', 0.8495546579360962),
 ('satisfying', 0.8487522602081299),
 ('delicious', 0.8262060284614563),
 ('filling', 0.8169576525688171),
 ('flavorful', 0.7985885739326477),
 ('addicting', 0.7648870944976807),
 ('delish', 0.7550991177558899),
 ('nutritious', 0.7429736852645874),
 ('tasteful', 0.7322741746902466)]

In [60]:
count_vect_feat = count_vect.get_feature_names()
